In [11]:
import pygame
import random
import copy
import pprint

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


Card

Deck


Player

Board

Game

Test

In [18]:
cards_1 = [Card('red',11), Card('g', 11), Card('b', 11)]
cards_2 = [Card('red',11), Card('red', 12), Card('red', 13)]
cards_3 = [Card('red',11), Card('red', 12), Card('Joker', 30)]
game1 = Game()
print(game1.is_group(cards_1))
print(game1.is_group(cards_2))
print(game1.is_run(cards_1))
print(game1.is_run(cards_2))
print(game1.is_first_move_valid(cards_3))
print(f"cards_3 is run: {game1.is_run(cards_3)}")

cards_4 = [ Card('red',11), Card('red', 12), Card('red', 13),\
            Card('red',11), Card('g', 11),   Card('b', 11), \
           ]
cards_5 = [Card('red',11), Card('g', 11),   Card('b', 11), \
           Card('red',11), Card('red', 12), Card('red', 13), \
           Card('red',11), Card('red', 12), Card('red', 13)
           ]
cards_6 = [ Card('g',11), Card('g', 12), Card('g', 13),
            Card('red',11), Card('g', 11),   Card('b', 11), 
           ]
# print(game1.check_combinations(cards_6))
# print(game1.find_all_combinations(cards_6))
all_valid_combinations = game1.find_all_combinations(cards_6)
print("all the valid combinations:")
for combination in all_valid_combinations:
    print(combination)

True
False
False
True
True
cards_3 is run: True
all the valid combinations:
[[Card(g 11), Card(g 12), Card(g 13)], [Card(b 11), Card(g 11), Card(red 11)]]
[[Card(b 11), Card(g 11), Card(red 11)], [Card(g 11), Card(g 12), Card(g 13)]]


In [53]:
game1.board

[]

In [56]:
game = Game()

deck1 = game.deck
print(len(deck1.cards_pool))
print(deck1.cards_pool)

p1 = Player('player1')
p2 = Player('player2')
p3 = Player('player3')
p4 = Player('player4')
players = [p1, p2, p3, p4]
print(players)      
for player in players:
    print(f"{player} hands:{player.hands}")
    player.draw_cards(deck1)
    print(f"{player} hands:{player.hands}")
    print(f"{player} holds cards number: {len(player.hands)}")
    
print(len(deck1.cards_pool))

106
[Card(Blue 4), Card(Blue 2), Card(Green 13), Card(Blue 9), Card(Blue 2), Card(Blue 1), Card(Red 7), Card(Blue 3), Card(Yellow 2), Card(Green 2), Card(Green 12), Card(Yellow 6), Card(Blue 9), Card(Red 13), Card(Red 9), Card(Red 5), Card(Yellow 12), Card(Blue 7), Card(Yellow 9), Card(Green 8), Card(Red 2), Card(Green 6), Card(Green 13), Card(Green 6), Card(Green 4), Card(Yellow 5), Card(Yellow 13), Card(Yellow 9), Card(Red 5), Card(Yellow 10), Card(Yellow 12), Card(Joker 30), Card(Red 3), Card(Joker 30), Card(Red 3), Card(Blue 1), Card(Red 8), Card(Yellow 3), Card(Red 7), Card(Red 2), Card(Blue 5), Card(Blue 6), Card(Blue 4), Card(Red 6), Card(Red 4), Card(Blue 11), Card(Blue 8), Card(Blue 3), Card(Red 9), Card(Red 4), Card(Red 10), Card(Yellow 10), Card(Yellow 11), Card(Yellow 7), Card(Green 1), Card(Green 2), Card(Yellow 4), Card(Blue 13), Card(Yellow 4), Card(Blue 12), Card(Green 11), Card(Red 12), Card(Yellow 8), Card(Green 8), Card(Yellow 6), Card(Yellow 5), Card(Green 10), Card

In [57]:
p1.hands.sort(key=lambda tile: (tile.number, tile.colour))
p1.hands

[Card(Red 1),
 Card(Yellow 1),
 Card(Yellow 3),
 Card(Green 5),
 Card(Blue 6),
 Card(Blue 7),
 Card(Green 7),
 Card(Yellow 8),
 Card(Blue 11),
 Card(Yellow 11),
 Card(Green 12),
 Card(Red 12),
 Card(Blue 13),
 Card(Red 13)]

In [46]:
game = Game()
game.first_move(p1, [[p1.hands[9], p1.hands[11], p1.hands[13]], ])
game.board

move success


[[Card(Yellow 10), Card(Yellow 11), Card(Joker 30)]]

In [47]:
p1.hands

[Card(Green 1),
 Card(Blue 2),
 Card(Green 2),
 Card(Red 3),
 Card(Green 5),
 Card(Blue 6),
 Card(Red 7),
 Card(Green 9),
 Card(Red 10),
 Card(Green 11),
 Card(Red 13)]

In [48]:
game.board

[[Card(Yellow 10), Card(Yellow 11), Card(Joker 30)]]